# Foreign Excange Prediction: Data Wrangling

###  In this notebook I am preparing data for my foreign exchange prediction project. </n>
#### I am utilizing the Alpha Vantage API to obtain pricing data from the past 100 days for the top ten most traded currencies:
* USD
* EUR
* JPY
* GBP
* AUD
* CHF
* CAD
* HKD
* SEK
* NZD </n>
#### Each section of the notebook will include some notes explaining what I am doing.

Import required tools for collecting and cleaning the data: <br>
* Some tools are more commonly used
* Some tools are specific to the Alpha Vantage API

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv('secret')

In [2]:
import json
import requests
import collections
import pandas as pd
import numpy as np
from ast import literal_eval
import datetime
import pickle
import time

In [3]:
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries

import asyncio
from alpha_vantage.async_support.timeseries import TimeSeries

Obtain forex data from the Alpha Vantage API: <br>
* I am obtaining pricing data for the past 100 days for pairings of the top ten most traded currencies <br>
* For each pairing I am taking only the data required into a pandas dataframe, adding a column for "to" and "from" symbol to keep track of each currency, and then concatenating the dataframes into a large dataframe for easier use

* Assign API call to a variable "app"

In [4]:
app = ForeignExchange(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')

* Create empty dataframe

In [5]:
all_pairs = pd.DataFrame()

Here we initialize two lists:
* CURRENCY_1: the currency we are converting FROM
* CURRENCY_2: the currency we are converting TO

In [6]:
CURRENCY_1 = ('USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CHF', 'CAD', 'HKD', 'SEK', 'NZD',)
CURRENCY_2 = ('USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CHF', 'CAD', 'HKD', 'SEK', 'NZD',)

Create the for loop:
* Double for loop to iterate through CURRENCY_1 and CURRENCY_2
* If statement to be sure that currency is not converted to itself
* Restrict data to usable part returned by API
* Create columns for "to symbol" and "from symbol"
* Rename columns for ease of use
* Convert to datetime, sort index
* Add features
* Reorder columns
* Concatenate to main dataframe

In [7]:
for c1 in CURRENCY_1:
  for c2 in CURRENCY_2:
    if c1 != c2:
        currconv = app.get_currency_exchange_daily(c1, c2)
        time.sleep(15)
        currconv1 = currconv[0]
        currconv1['to symbol'] = c2
        currconv1['from symbol'] = c1

        currconv1.rename(columns={'1. open': 'open'}, inplace=True)
        currconv1.rename(columns={'2. high': 'high'}, inplace=True)
        currconv1.rename(columns={'3. low': 'low'}, inplace=True)
        currconv1.rename(columns={'4. close': 'close'}, inplace=True)

        pd.to_datetime(currconv1.index)
        currconv1 = currconv1.sort_index()

        currconv1['time'] = np.arange(len(currconv1.index))

        currconv1['lag_1'] = currconv1['close'].shift(1)
        currconv1['lag_2'] = currconv1['close'].shift(2)
        currconv1['lag_3'] = currconv1['close'].shift(3)
        currconv1['lag_4'] = currconv1['close'].shift(4)
        currconv1['lag_5'] = currconv1['close'].shift(5)
        currconv1['lag_6'] = currconv1['close'].shift(6)
        currconv1['lag_7'] = currconv1['close'].shift(7)
        currconv1['lag_8'] = currconv1['close'].shift(8)
        currconv1['lag_9'] = currconv1['close'].shift(9)
        currconv1['lag_10'] = currconv1['close'].shift(10)
        currconv1['lag_11'] = currconv1['close'].shift(11)
        currconv1['lag_12'] = currconv1['close'].shift(12)
        currconv1['lag_13'] = currconv1['close'].shift(13)
        currconv1['lag_14'] = currconv1['close'].shift(14)

        currconv1['rolling_mean'] = currconv1['close'].rolling(window=7).mean()
        currconv1['expanding_mean'] = currconv1['close'].expanding(2).mean()

        currconv1 = currconv1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'rolling_mean', 'expanding_mean', 'time']]

        all_pairs = pd.concat([all_pairs, currconv1])

* View dataframe

In [8]:
all_pairs

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,...,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,,,,,,
2022-08-16,USD,EUR,0.9841,0.9876,0.9806,0.9829,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-08-17,USD,EUR,0.9832,0.9854,0.9800,0.9823,0.9829,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.982600,1
2022-08-18,USD,EUR,0.9821,0.9917,0.9807,0.9910,0.9823,0.9829,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.985400,2
2022-08-19,USD,EUR,0.9910,0.9965,0.9903,0.9956,0.9910,0.9823,0.9829,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.987950,3
2022-08-22,USD,EUR,0.9959,1.0072,0.9951,1.0053,0.9956,0.9910,0.9823,0.9829,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.991420,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,NZD,SEK,6.5672,6.6220,6.5497,6.5662,6.6114,6.5779,6.5383,6.5459,...,6.5584,6.5580,6.5884,6.5914,6.6039,6.5809,6.5933,6.578886,6.468579,96
2022-12-28,NZD,SEK,6.6099,6.6487,6.5856,6.6069,6.5662,6.6114,6.5779,6.5383,...,6.5479,6.5584,6.5580,6.5884,6.5914,6.6039,6.5809,6.579071,6.469991,97
2022-12-29,NZD,SEK,6.6321,6.6347,6.5728,6.6313,6.6069,6.5662,6.6114,6.5779,...,6.6056,6.5479,6.5584,6.5580,6.5884,6.5914,6.6039,6.582557,6.471620,98


* Save to pickle

In [9]:
all_pairs.to_pickle("all_pairs.pkl")

In [16]:
currconv = app.get_currency_exchange_daily('SEK', 'HKD')
currconv1 = currconv[0]

In [17]:
currconv1

,1. open,2. high,3. low,4. close
date,,,,
2023-01-02,0.7462,0.7468,0.7426,0.7428


In [18]:
ALPHA_VANTAGE_API_KEY

'D17LRLUSP9DEQ7DP'

In [ ]:
url = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=SEK&to_symbol=HKD&apikey=D17LRLUSP9DEQ7DP'
r = requests.get(url)
data = r.json()

In [20]:
data

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'SEK',
  '3. To Symbol': 'HKD',
  '4. Output Size': 'Compact',
  '5. Last Refreshed': '2023-01-02 19:30:00',
  '6. Time Zone': 'UTC'},
 'Time Series FX (Daily)': {'2023-01-02': {'1. open': '0.74620',
   '2. high': '0.74680',
   '3. low': '0.74260',
   '4. close': '0.74310'}}}